In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
from unicodedata import normalize
import pandas as pd
import json

In [2]:
job = input("Введите профессию: ").encode('utf-8')

Введите профессию: менеджер


In [3]:
url = 'https://barnaul.hh.ru/search/vacancy'

In [4]:
page = 0

In [5]:
params = {'text': job,
          'clusters': 'true',
          'ored_clusters': 'true',
          'enable_snippets': 'true',
          'salary': '',
          'page': page}

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [6]:
response = requests.get(url, params=params, headers=headers)

In [7]:
dom = bs(response.text, 'html.parser')

In [8]:
vacantions = dom.find_all('a', {'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'})

In [9]:
salaries = dom.find_all('span', {'data-qa': "vacancy-serp__vacancy-compensation"})

In [10]:
vacantion_list = []

In [11]:
if response.ok and vacantions:
    for vacantion in vacantions:
        vacantion_data = {}
        link = vacantion['href']
        name = vacantion.string
        vacantion_data['name'] = name
        vacantion_data['link'] = link
        vacantion_list.append(vacantion_data)

In [12]:
len(vacantions), len(salaries), len(vacantion_list)

(20, 20, 20)

In [13]:
for salary in salaries:
    salary_data = {}
    wage = salary.text
    stroka = wage.replace("\u202f", "")
    nums = re.findall(r'\d+', stroka)
    nums = [int(i) for i in nums]
    valuta = re.findall(r'\D+$', stroka)[0]
    valuta = normalize('NFKD', valuta)
    #salary_data['valuta'] = valuta
    
    if len(nums) == 2:
        salary_data['salary_min'] = nums[0]
        salary_data['salary_max'] = nums[1]
        salary_data['valuta'] = valuta
    elif len(nums) == 1 and stroka.find('от') == -1:
        salary_data['salary_min'] = None
        salary_data['salary_max'] = nums[0]
        salary_data['valuta'] = valuta
    elif len(nums) == 1 and stroka.find('до') == -1:
        salary_data['salary_min'] = nums[0]
        salary_data['salary_max'] = None
        salary_data['valuta'] = valuta
    else:
        salary_data['salary_min'] = None
        salary_data['salary_max'] = None
        salary_data['valuta'] = valuta
    
    vacantion_list.append(salary_data)

In [14]:
#salaries

In [15]:
#vacantion_list

In [16]:
for i in range(int(len(vacantion_list))):
    if i < int((len(vacantion_list))/2):
        vacantion_list[i].update(vacantion_list[int(i+(int(len(vacantion_list)))/2)])

In [17]:
vacantion_list = vacantion_list[:int(len(vacantion_list)/2)]

In [18]:
len(vacantion_list)

20

In [19]:
vacantion_list

[{'name': 'Менеджер по продажам (удаленно)',
  'link': 'https://barnaul.hh.ru/vacancy/49200200?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'salary_min': 65000,
  'salary_max': 125000,
  'valuta': ' руб.'},
 {'name': 'Менеджер по работе с клиентами',
  'link': 'https://barnaul.hh.ru/vacancy/49141640?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'salary_min': 1700,
  'salary_max': 2500,
  'valuta': ' бел. руб.'},
 {'name': 'Менеджер по продажам',
  'link': 'https://barnaul.hh.ru/vacancy/46875758?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'salary_min': 80000,
  'salary_max': 250000,
  'valuta': ' руб.'},
 {'name': 'Менеджер по продажам',
  'link': 'https://barnaul.hh.ru/vacancy/49172204?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'salary_min': 50000,
  'salary_max': 120000,
  'valuta': ' руб.'},
 {'name': 'Менеджер',
  'link': 'http

In [20]:
with open('2.1.json', 'w') as write_f:
    json.dump(vacantion_list, write_f)

In [38]:
df = pd.DataFrame(vacantion_list)

In [39]:
pd.set_option('display.max_columns', None)

In [42]:
pd.set_option('max_colwidth', 110)

In [41]:
df

,name,link,salary_min,salary_max,valuta
0,Менеджер по продажам (удаленно),https://barnaul.hh.ru/vacancy/49200200?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,65000.0,125000.0,руб.
1,Менеджер по работе с клиентами,https://barnaul.hh.ru/vacancy/49141640?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,1700.0,2500.0,бел. руб.
2,Менеджер по продажам,https://barnaul.hh.ru/vacancy/46875758?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,80000.0,250000.0,руб.
3,Менеджер по продажам,https://barnaul.hh.ru/vacancy/49172204?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,50000.0,120000.0,руб.
4,Менеджер,https://barnaul.hh.ru/vacancy/49165784?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,32000.0,NaN,руб.
5,Менеджер,https://barnaul.hh.ru/vacancy/49076668?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,110000.0,NaN,руб.
6,Менеджер по продажам,https://barnaul.hh.ru/vacancy/49071704?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,80000.0,200000.0,руб.
7,Менеджер по работе с клиентами,https://barnaul.hh.ru/vacancy/48889883?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,4000.0,8000.0,USD
8,Менеджер по работе с клиентами,https://barnaul.hh.ru/vacancy/49165144?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,120000.0,1200000.0,руб.
9,Менеджер по работе с клиентами,https://barnaul.hh.ru/vacancy/49090066?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80,12000.0,75000.0,руб.
